### Bag of word
> https://docs.google.com/presentation/d/1iLjWK9AQszJUSHxqDetBn5PXJ8SsmETcvCTNU2EGblE/edit#slide=id.p

目的: 處理不同長度的句子，但能表示字出現的頻率

#### Bag of word到TF-IDF
> https://taweihuang.hpd.io/2017/03/01/tfidf/

In [1]:
import pandas as pd
import numpy as np

Accept = pd.read_excel('ICLR_accepted.xlsx')
Reject = pd.read_excel('ICLR_rejected.xlsx')
Accept.head()

,Unnamed: 0,0
0,0,Minimal-Entropy Correlation Alignment for Unsu...
1,1,Large Scale Optimal Transport and Mapping Esti...
2,2,TRUNCATED HORIZON POLICY SEARCH: COMBINING REI...
3,3,Model-Ensemble Trust-Region Policy Optimization
4,4,A Neural Representation of Sketch Drawings


In [22]:
Accept.iloc[50:][0].head()

50    Unsupervised Representation Learning by Predic...
51    Emergent Communication in a Multi-Modal, Multi...
52    FastGCN: Fast Learning with Graph Convolutiona...
53    Emergent Translation in Multi-Agent Communication
54    An efficient framework for learning sentence r...
Name: 0, dtype: object

0是reject

In [85]:
X_train = pd.concat([Accept.iloc[50:][0], Reject.iloc[50:][0]]).reset_index(drop=True)
df1 = pd.DataFrame(np.ones(532))
df2 = pd.DataFrame(np.zeros(703))
y_train = pd.concat([df1, df2]).reset_index(drop=True)

X_test = pd.concat([Accept.iloc[:50][0], Reject.iloc[:50][0]]).reset_index(drop=True)
df3 = pd.DataFrame(np.ones(50))
df4 = pd.DataFrame(np.zeros(50))
y_test = pd.concat([df3, df4]).reset_index(drop=True)

In [14]:
# 參數設定
max_len = 30

建立字典/ 手刻字典

In [86]:
# 大寫轉小寫
X_train = X_train.str.lower()
X_test = X_test.str.lower()
# 建立字典
set = [] # 紀錄有哪些不同的單字
sentence_dict = {}

for index, row in X_train.iteritems():
    # 從句子切割單字出來
    for word in row.split(' '):
        if word not in set:
            set.append(word)
            if sorted(sentence_dict.keys()):
                sentence_dict[sorted(sentence_dict.keys())[-1]+1] = word
            else:
                sentence_dict[1] = word

# 手動轉換句子
for index, row in X_train.iteritems():
    num_sentence = []
    for word in row.split(' '):
        # 找到該單字在字典裡的 id
        num_sentence.append(list(sentence_dict.keys())[list(sentence_dict.values()).index(word)])
    X_train[index] = num_sentence
    
for index, row in X_test.iteritems():
    num_sentence = []
    for word in row.split(' '):
        try:
            num_sentence.append(list(sentence_dict.keys())[list(sentence_dict.values()).index(word)])
        except:
            # 字不存在就忽略
            continue
    X_test[index] = num_sentence

In [87]:
# 自動補齊長度
x_train_seq = sequence.pad_sequences(X_train, maxlen=max_len)
x_test_seq  = sequence.pad_sequences(X_test,  maxlen=max_len)

In [75]:
 X_train.head()

0                      [0, 1, 2, 3, 4, 5, 6]
1              [7, 8, 9, 10, 11, 12, 13, 14]
2    [15, 16, 2, 17, 18, 19, 20, 21, 22, 23]
3                          [7, 24, 9, 25, 8]
4                [26, 27, 28, 29, 2, 30, 31]
Name: 0, dtype: object

In [71]:
# 取得某個單字的dictnary id
list(sentence_dict.keys())[list(sentence_dict.values()).index('scale')]

82

In [60]:
sentence_dict = {1: 'haha', 0: 'XD'}
# 排列後找最後一個
sorted(sentence_dict.keys())[-1]

1

In [52]:
X_train.head().str.split(' ')

0    [unsupervised, representation, learning, by, p...
1    [emergent, communication, in, a, multi-modal,,...
2    [fastgcn:, fast, learning, with, graph, convol...
3    [emergent, translation, in, multi-agent, commu...
4    [an, efficient, framework, for, learning, sent...
Name: 0, dtype: object

底下這個cell是自動建立字典

In [26]:
from keras.preprocessing.text import Tokenizer
# 自動轉小寫
token = Tokenizer(lower=True)
token.fit_on_texts(X_train)
# 根據字典將句子轉換成數字
X_train_seq = token.texts_to_sequences(X_train)
X_test_seq = token.texts_to_sequences(X_test)

統一長度

In [38]:
from keras.preprocessing import sequence
x_train_seq = sequence.pad_sequences(X_train_seq, maxlen=max_len)
x_test_seq  = sequence.pad_sequences(X_test_seq,  maxlen=max_len)

In [34]:
len(token.word_index)

2137

### model

In [93]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import CuDNNLSTM
from keras.layers.embeddings import Embedding

model_LSTM = Sequential()

# Embedding
# model_LSTM.add(Embedding(output_dim=64, input_dim=len(token.word_index), input_length=max_len))
model_LSTM.add(Embedding(output_dim=64, input_dim=len(sentence_dict), input_length=max_len))
model_LSTM.add(CuDNNLSTM(64, input_shape=(max_len, 1), return_sequences=True))
# model_LSTM.add(CuDNNLSTM(50, return_sequences=True))
# model_LSTM.add(CuDNNLSTM(40, return_sequences=True))
# model_LSTM.add(CuDNNLSTM(30, return_sequences=True))
# model_LSTM.add(CuDNNLSTM(20, return_sequences=True))
model_LSTM.add(CuDNNLSTM(40, return_sequences=False))

model_LSTM.add(Dropout(0.25))

model_LSTM.add(Dense(40, activation='relu'))
model_LSTM.add(Dense(20, activation='relu'))
model_LSTM.add(Dense(2, activation='softmax'))

model_LSTM.summary()

model_LSTM.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 30, 64)            154944    
_________________________________________________________________
cu_dnnlstm_19 (CuDNNLSTM)    (None, 30, 64)            33280     
_________________________________________________________________
cu_dnnlstm_20 (CuDNNLSTM)    (None, 40)                16960     
_________________________________________________________________
dropout_4 (Dropout)          (None, 40)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 40)                1640      
_________________________________________________________________
dense_11 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 42        
Total para

In [94]:
model_LSTM.fit(x_train_seq, y_train,batch_size=20, epochs=10,verbose=2, validation_split=0.2)

Train on 988 samples, validate on 247 samples
Epoch 1/30
 - 1s - loss: 0.6918 - acc: 0.5162 - val_loss: 0.8290 - val_acc: 0.0000e+00
Epoch 2/30
 - 0s - loss: 0.6895 - acc: 0.5374 - val_loss: 0.7434 - val_acc: 0.0000e+00
Epoch 3/30
 - 0s - loss: 0.6277 - acc: 0.6528 - val_loss: 0.6884 - val_acc: 0.5911
Epoch 4/30
 - 0s - loss: 0.3193 - acc: 0.8725 - val_loss: 1.3558 - val_acc: 0.3603
Epoch 5/30
 - 0s - loss: 0.1496 - acc: 0.9504 - val_loss: 1.2833 - val_acc: 0.5020
Epoch 6/30
 - 0s - loss: 0.0928 - acc: 0.9686 - val_loss: 1.5695 - val_acc: 0.5101
Epoch 7/30
 - 0s - loss: 0.0745 - acc: 0.9777 - val_loss: 2.1263 - val_acc: 0.4291
Epoch 8/30
 - 0s - loss: 0.0526 - acc: 0.9808 - val_loss: 1.5801 - val_acc: 0.5911
Epoch 9/30
 - 0s - loss: 0.0350 - acc: 0.9899 - val_loss: 2.0657 - val_acc: 0.5506
Epoch 10/30
 - 0s - loss: 0.0262 - acc: 0.9929 - val_loss: 3.0906 - val_acc: 0.4291
Epoch 11/30
 - 0s - loss: 0.0125 - acc: 0.9970 - val_loss: 3.7829 - val_acc: 0.4130
Epoch 12/30
 - 0s - loss: 0.010

### test model

In [92]:
model_LSTM.evaluate(x_test_seq, y_test)

100/100 [==============================] - 0s 463us/step


[2.6161777496337892, 0.65]

#### train的validation達到60趴, test上反而比較低

In [95]:
model_LSTM.evaluate(x_test_seq, y_test)

100/100 [==============================] - 0s 165us/step


[3.4139516735076905, 0.57]